# WKT To AC Node
1 -> 1

Converteert [well-known text](https://en.wikipedia.org/wiki/Well-known_text#Geometric_objects) (WKT) naar AC format (`x,y:x,y@x,y:x,y`). Beide formats hanteren de punt (`.`) als decimale punt.

De `output_table` kan in KNIME worden doorgegeven naar bijvoorbeeld de *CSV Writer* node.

In [7]:
###
### WKT TO AC NODE
###
def wktToAc(wkt):
    ac = wkt\
        .replace('MULTIPOLYGON (((', '').replace(')))', '').replace(')), ((', '@')\
        .replace('POLYGON ((', '').replace('))', '').replace('), (', '@')\
        .replace(', ', ':').replace(' ', ',')
    return ac

# copy input table and convert wkt to AC into new column
output_table = input_table.copy()
output_table['AC'] = input_table.wkt.apply(wktToAc)
print('Shape output_table:', output_table.shape)
print('Preview output_table:\n', output_table.head())

Shape output_table: (12, 4)
Preview output_table:
    id     provincien                                                wkt  \
0   1  Noord-Holland  MULTIPOLYGON (((5.16595803513613 53.0011795601...   
1   2      Groningen  MULTIPOLYGON (((6.287547703816826 53.342509935...   
2   3     Overijssel  MULTIPOLYGON (((6.110121318419511 52.441553119...   
3   4        Zeeland  MULTIPOLYGON (((3.83923160158486 51.7592703536...   
4   5      Friesland  MULTIPOLYGON (((6.191971949520441 53.412179416...   

                                                  AC  
0  5.16595803513613,53.00117956010834:5.166302802...  
1  6.287547703816826,53.34250993573775:6.28473742...  
2  6.110121318419511,52.44155311970717:6.10958144...  
3  3.83923160158486,51.75927035368181:3.841869950...  
4  6.191971949520441,53.41217941682984:6.19167794...  


## Load some input data to test the cells above

In [6]:
###
### SOURCE - ONLY FOR PREPARATION
###
from os.path import join
folder = '/home/ab/i/Open-data/shapefiles/shp-provincie'
filename = 'provincie-grenzen.shp'

# geopandas (geometry) to pandas (wkt)
import geopandas as gpd
import pandas as pd
from shapely import wkt as WKT
def gdfToDf(gdf):
    df = pd.DataFrame(gdf, copy=True)
    df['wkt'] = gdf.geometry.apply(lambda wkt: WKT.dumps(wkt, trim=True))
    df.drop(columns='geometry', inplace=True)
    return df

# read the file
gdf = gpd.read_file( join(folder, filename) )
gdf = gdf.to_crs(epsg=4326)    # WGS84

# output
output_table = gdfToDf(gdf)

# copy output to input
input_table = output_table.copy()